In [1]:
# Imports
import pandas as pd
import numpy as np

from IPython.display import clear_output

In [2]:
df = pd.read_csv("data/Preprocessed_Data.csv")

/tmp/ipykernel_10843/1958202050.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/Preprocessed_Data.csv")


In [3]:
df

,order_id,R_Lat,R_Lon,C_Lat,C_Lon,order_pickedup_time,order_delivered_time,Distance (km),Delivery_Hour,State,Delivery_Time (sec)
0,PRD_ZMT_2960011,28.468983,77.063728,28.449929,77.071230,2016-04-23 09:30:20,2016-04-23 09:57:04,2.242045,9,Haryana,1604.0
1,PRD_ZMT_2964555,28.468983,77.063728,28.454522,77.066637,2016-04-23 11:05:32,2016-04-23 11:27:10,1.632941,11,Haryana,1298.0
2,PRD_ZMT_2968945,28.468983,77.063728,28.461570,77.067590,2016-04-23 14:49:07,2016-04-23 15:07:18,0.906626,14,Haryana,1091.0
3,PRD_ZMT_2976663,28.468983,77.063728,28.455738,77.058401,2016-04-23 16:43:14,2016-04-23 17:16:12,1.562127,16,Haryana,1978.0
4,PRD_ZMT_2985948,28.468983,77.063728,28.440068,77.083967,2016-04-24 07:54:14,2016-04-24 08:16:29,3.775236,7,Haryana,1335.0
...,...,...,...,...,...,...,...,...,...,...,...
237368,PRD_ZMT_8457746,28.532886,77.207941,28.503037,77.201892,2016-09-25 16:49:02,2016-09-25 17:18:55,3.371266,16,NaN,1793.0
237369,PRD_ZMT_8458181,22.544237,88.352148,22.560074,88.369037,2016-09-25 17:07:48,2016-09-25 17:22:04,2.471663,17,NaN,856.0
237370,PRD_ZMT_8458897,19.122455,72.916411,19.104956,72.918557,2016-09-25 17:22:39,2016-09-25 17:43:55,1.958820,17,NaN,1276.0
237371,PRD_ZMT_8458955,17.450286,78.379158,17.444102,78.391155,2016-09-25 17:07:15,2016-09-25 17:26:16,1.446523,17,NaN,1141.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237373 entries, 0 to 237372
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              237373 non-null  object 
 1   R_Lat                 237373 non-null  float64
 2   R_Lon                 237373 non-null  float64
 3   C_Lat                 237373 non-null  float64
 4   C_Lon                 237373 non-null  float64
 5   order_pickedup_time   237373 non-null  object 
 6   order_delivered_time  237373 non-null  object 
 7   Distance (km)         237373 non-null  float64
 8   Delivery_Hour         237373 non-null  int64  
 9   State                 42301 non-null   object 
 10  Delivery_Time (sec)   237373 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 19.9+ MB


In [5]:
# For incomplete data
df = df[:50000]
df

,order_id,R_Lat,R_Lon,C_Lat,C_Lon,order_pickedup_time,order_delivered_time,Distance (km),Delivery_Hour,State,Delivery_Time (sec)
0,PRD_ZMT_2960011,28.468983,77.063728,28.449929,77.071230,2016-04-23 09:30:20,2016-04-23 09:57:04,2.242045,9,Haryana,1604.0
1,PRD_ZMT_2964555,28.468983,77.063728,28.454522,77.066637,2016-04-23 11:05:32,2016-04-23 11:27:10,1.632941,11,Haryana,1298.0
2,PRD_ZMT_2968945,28.468983,77.063728,28.461570,77.067590,2016-04-23 14:49:07,2016-04-23 15:07:18,0.906626,14,Haryana,1091.0
3,PRD_ZMT_2976663,28.468983,77.063728,28.455738,77.058401,2016-04-23 16:43:14,2016-04-23 17:16:12,1.562127,16,Haryana,1978.0
4,PRD_ZMT_2985948,28.468983,77.063728,28.440068,77.083967,2016-04-24 07:54:14,2016-04-24 08:16:29,3.775236,7,Haryana,1335.0
...,...,...,...,...,...,...,...,...,...,...,...
49995,PRD_ZMT_4598381,28.541147,77.155150,28.512131,77.177924,2016-06-22 12:17:58,2016-06-22 12:26:39,3.919197,12,NaN,521.0
49996,PRD_ZMT_4598429,28.564568,77.199208,28.558845,77.200506,2016-06-22 11:57:19,2016-06-22 12:44:10,0.648872,11,NaN,2811.0
49997,PRD_ZMT_4598618,28.468983,77.063728,28.476535,77.060448,2016-06-22 12:17:56,2016-06-22 17:22:57,0.898864,12,NaN,18301.0
49998,PRD_ZMT_4598662,17.440667,78.392478,17.447236,78.387076,2016-06-22 12:03:17,2016-06-22 12:10:55,0.928401,12,NaN,458.0


In [6]:
df["Delivery_Time (min)"] = df["Delivery_Time (sec)"] / 60

/tmp/ipykernel_10843/601385811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Delivery_Time (min)"] = df["Delivery_Time (sec)"] / 60


In [7]:
df.drop(columns="Delivery_Time (sec)", inplace=True)

/tmp/ipykernel_10843/1944161867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns="Delivery_Time (sec)", inplace=True)


# These many people have ordered and received in the restaurant itself. Probably, they entered the restaurant and checked online price & found it cheaper. They must be engineers 💯 😎

In [8]:
eng = len(df[df["Distance (km)"] == 0])
print(f"{eng} ({eng / len(df)}%) people have received the order in the restaurant itself!!")

996 (0.01992%) people have received the order in the restaurant itself!!


In [9]:
df

,order_id,R_Lat,R_Lon,C_Lat,C_Lon,order_pickedup_time,order_delivered_time,Distance (km),Delivery_Hour,State,Delivery_Time (min)
0,PRD_ZMT_2960011,28.468983,77.063728,28.449929,77.071230,2016-04-23 09:30:20,2016-04-23 09:57:04,2.242045,9,Haryana,26.733333
1,PRD_ZMT_2964555,28.468983,77.063728,28.454522,77.066637,2016-04-23 11:05:32,2016-04-23 11:27:10,1.632941,11,Haryana,21.633333
2,PRD_ZMT_2968945,28.468983,77.063728,28.461570,77.067590,2016-04-23 14:49:07,2016-04-23 15:07:18,0.906626,14,Haryana,18.183333
3,PRD_ZMT_2976663,28.468983,77.063728,28.455738,77.058401,2016-04-23 16:43:14,2016-04-23 17:16:12,1.562127,16,Haryana,32.966667
4,PRD_ZMT_2985948,28.468983,77.063728,28.440068,77.083967,2016-04-24 07:54:14,2016-04-24 08:16:29,3.775236,7,Haryana,22.250000
...,...,...,...,...,...,...,...,...,...,...,...
49995,PRD_ZMT_4598381,28.541147,77.155150,28.512131,77.177924,2016-06-22 12:17:58,2016-06-22 12:26:39,3.919197,12,NaN,8.683333
49996,PRD_ZMT_4598429,28.564568,77.199208,28.558845,77.200506,2016-06-22 11:57:19,2016-06-22 12:44:10,0.648872,11,NaN,46.850000
49997,PRD_ZMT_4598618,28.468983,77.063728,28.476535,77.060448,2016-06-22 12:17:56,2016-06-22 17:22:57,0.898864,12,NaN,305.016667
49998,PRD_ZMT_4598662,17.440667,78.392478,17.447236,78.387076,2016-06-22 12:03:17,2016-06-22 12:10:55,0.928401,12,NaN,7.633333


# Let's analyze delivery speed

In [10]:
df["Delivery_Speed (min/km)"] = 1

for i in range(len(df)):
    if df.loc[i, "Distance (km)"] == 0:
        df.loc[i, "Delivery_Speed (min/km)"] = df.loc[i, "Delivery_Time (min)"]
    else:
        df.loc[i, "Delivery_Speed (min/km)"] = df.loc[i, "Delivery_Time (min)"] / df.loc[i, "Distance (km)"]

/tmp/ipykernel_10843/270017328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Delivery_Speed (min/km)"] = 1
/tmp/ipykernel_10843/270017328.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '11.923638034356124' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, "Delivery_Speed (min/km)"] = df.loc[i, "Delivery_Time (min)"] / df.loc[i, "Distance (km)"]


In [11]:
df

,order_id,R_Lat,R_Lon,C_Lat,C_Lon,order_pickedup_time,order_delivered_time,Distance (km),Delivery_Hour,State,Delivery_Time (min),Delivery_Speed (min/km)
0,PRD_ZMT_2960011,28.468983,77.063728,28.449929,77.071230,2016-04-23 09:30:20,2016-04-23 09:57:04,2.242045,9,Haryana,26.733333,11.923638
1,PRD_ZMT_2964555,28.468983,77.063728,28.454522,77.066637,2016-04-23 11:05:32,2016-04-23 11:27:10,1.632941,11,Haryana,21.633333,13.248076
2,PRD_ZMT_2968945,28.468983,77.063728,28.461570,77.067590,2016-04-23 14:49:07,2016-04-23 15:07:18,0.906626,14,Haryana,18.183333,20.056042
3,PRD_ZMT_2976663,28.468983,77.063728,28.455738,77.058401,2016-04-23 16:43:14,2016-04-23 17:16:12,1.562127,16,Haryana,32.966667,21.103701
4,PRD_ZMT_2985948,28.468983,77.063728,28.440068,77.083967,2016-04-24 07:54:14,2016-04-24 08:16:29,3.775236,7,Haryana,22.250000,5.893672
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,PRD_ZMT_4598381,28.541147,77.155150,28.512131,77.177924,2016-06-22 12:17:58,2016-06-22 12:26:39,3.919197,12,NaN,8.683333,2.215590
49996,PRD_ZMT_4598429,28.564568,77.199208,28.558845,77.200506,2016-06-22 11:57:19,2016-06-22 12:44:10,0.648872,11,NaN,46.850000,72.202241
49997,PRD_ZMT_4598618,28.468983,77.063728,28.476535,77.060448,2016-06-22 12:17:56,2016-06-22 17:22:57,0.898864,12,NaN,305.016667,339.335563
49998,PRD_ZMT_4598662,17.440667,78.392478,17.447236,78.387076,2016-06-22 12:03:17,2016-06-22 12:10:55,0.928401,12,NaN,7.633333,8.222021


## State wise average delivery speed

In [13]:
avg = df.groupby("State").mean("Delivery_Speed (min/km)").drop(
    columns=["R_Lat", "R_Lon", "C_Lat", "C_Lon", "Distance (km)", "Delivery_Hour", "Delivery_Time (min)"]).rename(columns={"Delivery_Speed (min/km)": "Average Delivery_Speed (min/km)"})

In [14]:
avg

,Average Delivery_Speed (min/km)
State,
Borno,0.000112
Delhi,79.619884
Haryana,84.506053
Karnataka,122.035415
Maharashtra,57.228198
Tamil Nadu,53.355196
Telangana,58.883546
West Bengal,33.337526


## Overall average delivery speed

In [15]:
df["Delivery_Speed (min/km)"].mean()

71.75995855269989